# Analisis Mendalam Data Pemesanan Hotel

## Bagian 1: Preprocessing Data

Notebook ini berisi langkah-langkah preprocessing data pemesanan hotel untuk persiapan analisis dan visualisasi. Preprocessing yang dilakukan meliputi penanganan missing values, pembersihan data, dan pembuatan fitur baru yang akan berguna untuk analisis lanjutan.

## 1. Import Library dan Konfigurasi

In [1]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import os
import logging
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Konfigurasi visualisasi
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Konfigurasi logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('hotel_preprocessing')

# Direktori untuk menyimpan data
DATA_DIR = 'hotel_booking_data'
os.makedirs(DATA_DIR, exist_ok=True)

# Mengabaikan warning
import warnings
warnings.filterwarnings('ignore')

## 2. Memuat Data

Pada bagian ini, kita akan memuat dataset pemesanan hotel dari file CSV atau Excel. Fungsi `load_data` akan mendeteksi format file dan memuat data dengan cara yang sesuai.

In [2]:
def load_data(file_path):
    """
    Memuat dataset dari file CSV atau Excel.

    Parameters:
    -----------
    file_path : str
        Path ke file dataset

    Returns:
    --------
    pandas.DataFrame
        Dataset yang telah dimuat
    """
    logger.info(f"Memuat data dari {file_path}")

    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith(('.xls', '.xlsx')):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Format file tidak didukung. Gunakan CSV atau Excel.")

    logger.info(f"Data berhasil dimuat dengan {df.shape[0]} baris dan {df.shape[1]} kolom.")
    return df

### 2.1 Memuat Dataset

Sekarang kita akan memuat dataset pemesanan hotel. Pastikan file dataset tersedia di path yang ditentukan.

In [3]:
# Ganti path file sesuai dengan lokasi dataset Anda
file_path = 'hotel_booking_data/hotel_bookings.csv'

# Memuat dataset
try:
    hotel_df = load_data(file_path)
    print(f"Dataset berhasil dimuat dengan {hotel_df.shape[0]} baris dan {hotel_df.shape[1]} kolom.")
except FileNotFoundError:
    print(f"File tidak ditemukan: {file_path}")
    print("Silakan unduh dataset dan letakkan di path yang benar.")
    # Jika file tidak ditemukan, kita akan membuat contoh dataframe kosong untuk demonstrasi
    hotel_df = pd.DataFrame()

2025-05-02 09:20:05,061 - hotel_preprocessing - INFO - Memuat data dari hotel_booking_data/hotel_bookings.csv
2025-05-02 09:20:05,286 - hotel_preprocessing - INFO - Data berhasil dimuat dengan 119390 baris dan 32 kolom.


Dataset berhasil dimuat dengan 119390 baris dan 32 kolom.


### 2.2 Eksplorasi Awal Data

Sebelum melakukan preprocessing, kita akan melakukan eksplorasi awal untuk memahami struktur dan karakteristik data.

In [4]:
# Memeriksa struktur data
if not hotel_df.empty:
    print("\n=== INFORMASI DATASET ===")
    hotel_df.info()

    print("\n=== STATISTIK DESKRIPTIF ===")
    print(hotel_df.describe())

    print("\n=== SAMPEL DATA ===")
    display(hotel_df.head())

    print("\n=== MISSING VALUES ===")
    missing_values = hotel_df.isnull().sum()
    missing_percentage = (missing_values / len(hotel_df)) * 100
    missing_info = pd.DataFrame({
        'Missing Values': missing_values,
        'Percentage': missing_percentage
    })
    display(missing_info[missing_info['Missing Values'] > 0].sort_values('Percentage', ascending=False))


=== INFORMASI DATASET ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  in

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03



=== MISSING VALUES ===


,Missing Values,Percentage
company,112593,94.306893
agent,16340,13.686238
country,488,0.408744
children,4,0.003350


## 3. Penanganan Missing Values

Pada bagian ini, kita akan menangani missing values dalam dataset. Strategi penanganan missing values yang digunakan adalah:

1. **Menghapus kolom dengan missing values tinggi**: Kolom dengan persentase missing values yang sangat tinggi (>90%) akan dihapus karena tidak memberikan informasi yang cukup.
2. **Menghapus baris dengan missing values pada kolom penting**: Baris dengan missing values pada kolom penting seperti 'country' dan 'children' akan dihapus karena jumlahnya relatif sedikit.
3. **Mengisi missing values dengan nilai yang sesuai**: Missing values pada kolom lain akan diisi dengan nilai yang sesuai berdasarkan konteks bisnis.

In [5]:
def handle_missing_values(df):
    """
    Menangani missing values dalam dataset.

    Parameters:
    -----------
    df : pandas.DataFrame
        Dataset yang akan diproses

    Returns:
    --------
    pandas.DataFrame
        Dataset setelah penanganan missing values
    """
    logger.info("Menangani missing values...")

    # Memeriksa missing values
    missing_values = df.isnull().sum()
    missing_percentage = (missing_values / len(df)) * 100
    missing_info = pd.DataFrame({
        'Missing Values': missing_values,
        'Percentage': missing_percentage
    })
    logger.info(f"Missing values sebelum penanganan:\n{missing_info[missing_info['Missing Values'] > 0]}")

    # Hapus kolom company (94.31% missing)
    if 'company' in df.columns:
        df = df.drop('company', axis=1)
        logger.info("Kolom 'company' dihapus karena missing values >90%.")
        print("Kolom 'company' dihapus karena memiliki >90% missing values. Kolom ini tidak memberikan informasi yang cukup untuk analisis.")

    # Hapus baris dengan country kosong (0.41%)
    if 'country' in df.columns:
        country_null_count = df['country'].isnull().sum()
        if country_null_count > 0:
            df = df.dropna(subset=['country'])
            logger.info(f"Menghapus {country_null_count} baris dengan missing country (total {country_null_count/len(df):.2%}).")
            print(f"Menghapus {country_null_count} baris dengan missing country. Informasi negara asal tamu penting untuk analisis segmentasi pasar.")

    # Hapus baris dengan children kosong (0.003%)
    if 'children' in df.columns:
        children_null_count = df['children'].isnull().sum()
        if children_null_count > 0:
            df = df.dropna(subset=['children'])
            logger.info(f"Menghapus {children_null_count} baris dengan missing children (total {children_null_count/len(df):.2%}).")
            print(f"Menghapus {children_null_count} baris dengan missing children. Jumlahnya sangat kecil dan informasi ini penting untuk analisis komposisi tamu.")

    # Mengisi missing values untuk agent dengan 0 (menandakan tidak ada agen)
    if 'agent' in df.columns:
        agent_null_count = df['agent'].isnull().sum()
        if agent_null_count > 0:
            df['agent'] = df['agent'].fillna(0)
            logger.info(f"Mengisi {agent_null_count} missing values di kolom 'agent' dengan 0 (menandakan tidak ada agen).")
            print(f"Mengisi {agent_null_count} missing values di kolom 'agent' dengan 0. Nilai null pada kolom ini menandakan pemesanan tanpa agen.")

    # Mengisi missing values untuk meal dengan 'No Meal' (SC - Self Catering)
    if 'meal' in df.columns:
        meal_null_count = df['meal'].isnull().sum()
        if meal_null_count > 0:
            df['meal'] = df['meal'].fillna('SC')
            logger.info(f"Mengisi {meal_null_count} missing values di kolom 'meal' dengan 'SC' (Self Catering).")
            print(f"Mengisi {meal_null_count} missing values di kolom 'meal' dengan 'SC' (Self Catering). Asumsi: tamu tanpa paket makanan tertentu menggunakan opsi self-catering.")

    # Memeriksa missing values setelah penanganan
    missing_after = df.isnull().sum()
    logger.info(f"Missing values setelah penanganan:\n{missing_after[missing_after > 0]}")

    return df

### 3.1 Menerapkan Penanganan Missing Values

Sekarang kita akan menerapkan fungsi penanganan missing values pada dataset.

In [6]:
if not hotel_df.empty:
    hotel_df = handle_missing_values(hotel_df)

    print("\n=== MISSING VALUES SETELAH PENANGANAN ===")
    missing_after = hotel_df.isnull().sum()
    display(missing_after[missing_after > 0])

2025-05-02 09:20:05,527 - hotel_preprocessing - INFO - Menangani missing values...
2025-05-02 09:20:05,576 - hotel_preprocessing - INFO - Missing values sebelum penanganan:
          Missing Values  Percentage
children               4    0.003350
country              488    0.408744
agent              16340   13.686238
company           112593   94.306893
2025-05-02 09:20:05,594 - hotel_preprocessing - INFO - Kolom 'company' dihapus karena missing values >90%.
2025-05-02 09:20:05,626 - hotel_preprocessing - INFO - Menghapus 488 baris dengan missing country (total 0.41%).
2025-05-02 09:20:05,649 - hotel_preprocessing - INFO - Menghapus 4 baris dengan missing children (total 0.00%).
2025-05-02 09:20:05,652 - hotel_preprocessing - INFO - Mengisi 16004 missing values di kolom 'agent' dengan 0 (menandakan tidak ada agen).


Kolom 'company' dihapus karena memiliki >90% missing values. Kolom ini tidak memberikan informasi yang cukup untuk analisis.
Menghapus 488 baris dengan missing country. Informasi negara asal tamu penting untuk analisis segmentasi pasar.
Menghapus 4 baris dengan missing children. Jumlahnya sangat kecil dan informasi ini penting untuk analisis komposisi tamu.
Mengisi 16004 missing values di kolom 'agent' dengan 0. Nilai null pada kolom ini menandakan pemesanan tanpa agen.


2025-05-02 09:20:05,697 - hotel_preprocessing - INFO - Missing values setelah penanganan:
Series([], dtype: int64)



=== MISSING VALUES SETELAH PENANGANAN ===


Series([], dtype: int64)

## 4. Pembersihan Data

Pada bagian ini, kita akan membersihkan data dari nilai yang tidak valid atau inkonsisten. Langkah-langkah pembersihan data meliputi:

1. **Menghapus nilai undefined di distribution_channel**: Nilai 'Undefined' tidak memberikan informasi yang berguna dan dapat mengganggu analisis distribusi saluran pemesanan.
2. **Menghapus reservasi tanpa tamu**: Reservasi dengan total tamu = 0 kemungkinan merupakan kesalahan data atau pembatalan yang tidak tercatat dengan benar.
3. **Memperbaiki inkonsistensi antara is_canceled dan reservation_status**: Memastikan konsistensi data antara kolom is_canceled dan reservation_status untuk analisis pembatalan yang akurat.
4. **Memperbaiki nilai negatif di kolom numerik**: Nilai negatif untuk kolom seperti jumlah tamu atau durasi menginap tidak masuk akal secara logis dan kemungkinan merupakan kesalahan data.

In [7]:
def clean_data(df):
    """
    Membersihkan data dari nilai yang tidak valid atau inkonsisten.

    Parameters:
    -----------
    df : pandas.DataFrame
        Dataset yang akan dibersihkan

    Returns:
    --------
    pandas.DataFrame
        Dataset yang telah dibersihkan
    """
    logger.info("Membersihkan data...")

    # Menghapus nilai undefined di distribution_channel
    if 'distribution_channel' in df.columns:
        undefined_count = df[df['distribution_channel'] == 'Undefined'].shape[0]
        if undefined_count > 0:
            df = df[df['distribution_channel'] != 'Undefined']
            logger.info(f"Menghapus {undefined_count} baris dengan distribution_channel 'Undefined'.")
            print(f"Menghapus {undefined_count} baris dengan distribution_channel 'Undefined'.")
            print("Alasan: Nilai 'Undefined' tidak memberikan informasi yang berguna dan dapat mengganggu analisis distribusi saluran pemesanan.")

    # Menghapus reservasi tanpa tamu (total_guests = 0)
    if 'adults' in df.columns and 'children' in df.columns and 'babies' in df.columns:
        df['total_guests'] = df['adults'] + df['children'] + df['babies']
        no_guests_count = len(df[df['total_guests'] == 0])
        if no_guests_count > 0:
            df = df[df['total_guests'] != 0]
            logger.info(f"Menghapus {no_guests_count} reservasi tanpa tamu (total {no_guests_count/len(df):.2%}).")
            print(f"Menghapus {no_guests_count} reservasi tanpa tamu (total_guests = 0).")
            print("Alasan: Reservasi tanpa tamu kemungkinan merupakan kesalahan data atau pembatalan yang tidak tercatat dengan benar.")

    # Memperbaiki inkonsistensi antara is_canceled dan reservation_status
    if 'is_canceled' in df.columns and 'reservation_status' in df.columns:
        inconsistent_count = len(df[(df['is_canceled'] == 1) & (df['reservation_status'] != 'Canceled')])
        if inconsistent_count > 0:
            df.loc[df['is_canceled'] == 1, 'reservation_status'] = 'Canceled'
            logger.info(f"Memperbaiki {inconsistent_count} inkonsistensi antara is_canceled dan reservation_status.")
            print(f"Memperbaiki {inconsistent_count} inkonsistensi antara is_canceled dan reservation_status.")
            print("Alasan: Memastikan konsistensi data antara kolom is_canceled dan reservation_status untuk analisis pembatalan yang akurat.")

    # Memperbaiki nilai negatif di kolom numerik tertentu
    numeric_columns = ['adults', 'children', 'babies', 'stays_in_weekend_nights', 'stays_in_week_nights']
    for col in numeric_columns:
        if col in df.columns:
            negative_count = (df[col] < 0).sum()
            if negative_count > 0:
                df = df[df[col] >= 0]
                logger.info(f"Menghapus {negative_count} baris dengan nilai negatif di kolom '{col}'.")
                print(f"Menghapus {negative_count} baris dengan nilai negatif di kolom '{col}'.")
                print(f"Alasan: Nilai negatif untuk {col} tidak masuk akal secara logis dan kemungkinan merupakan kesalahan data.")

    return df

### 4.1 Menerapkan Pembersihan Data

Sekarang kita akan menerapkan fungsi pembersihan data pada dataset.

In [8]:
if not hotel_df.empty:
    hotel_df = clean_data(hotel_df)

    print("\n=== UKURAN DATASET SETELAH PEMBERSIHAN ===")
    print(f"Jumlah baris: {hotel_df.shape[0]}")
    print(f"Jumlah kolom: {hotel_df.shape[1]}")

2025-05-02 09:20:05,757 - hotel_preprocessing - INFO - Membersihkan data...
2025-05-02 09:20:05,785 - hotel_preprocessing - INFO - Menghapus 1 baris dengan distribution_channel 'Undefined'.


Menghapus 1 baris dengan distribution_channel 'Undefined'.
Alasan: Nilai 'Undefined' tidak memberikan informasi yang berguna dan dapat mengganggu analisis distribusi saluran pemesanan.


2025-05-02 09:20:05,807 - hotel_preprocessing - INFO - Menghapus 170 reservasi tanpa tamu (total 0.14%).
2025-05-02 09:20:05,817 - hotel_preprocessing - INFO - Memperbaiki 1202 inkonsistensi antara is_canceled dan reservation_status.


Menghapus 170 reservasi tanpa tamu (total_guests = 0).
Alasan: Reservasi tanpa tamu kemungkinan merupakan kesalahan data atau pembatalan yang tidak tercatat dengan benar.
Memperbaiki 1202 inkonsistensi antara is_canceled dan reservation_status.
Alasan: Memastikan konsistensi data antara kolom is_canceled dan reservation_status untuk analisis pembatalan yang akurat.

=== UKURAN DATASET SETELAH PEMBERSIHAN ===
Jumlah baris: 118727
Jumlah kolom: 32


## 5. Pembuatan Fitur Baru

Pada bagian ini, kita akan membuat fitur baru yang berguna untuk analisis. Fitur-fitur baru yang akan dibuat meliputi:

1. **Fitur tanggal**: Menggabungkan tanggal kedatangan menjadi satu kolom dan mengekstrak fitur waktu tambahan seperti bulan, hari, dan hari dalam seminggu.
2. **Fitur durasi menginap**: Menghitung total durasi menginap berdasarkan jumlah malam di akhir pekan dan hari kerja.
3. **Kategori lead time**: Mengelompokkan lead time (waktu antara pemesanan dan kedatangan) menjadi beberapa kategori untuk analisis yang lebih mudah.
4. **Fitur boolean**: Membuat fitur boolean untuk mempermudah analisis, seperti has_children, has_booking_changes, is_repeat_guest, dll.
5. **Fitur analisis tipe kamar**: Membuat fitur untuk analisis tipe kamar, seperti room_type_match, room_type_upgrade, dan room_type_downgrade.
6. **Fitur analisis asal tamu**: Membuat fitur untuk analisis asal tamu, seperti is_domestic dan region.

In [9]:
def create_features(df):
    """
    Membuat fitur baru yang berguna untuk analisis.

    Parameters:
    -----------
    df : pandas.DataFrame
        Dataset yang akan ditambahkan fitur baru

    Returns:
    --------
    pandas.DataFrame
        Dataset dengan fitur baru
    """
    logger.info("Membuat fitur baru...")

    # Menggabungkan tanggal kedatangan menjadi satu kolom
    if all(col in df.columns for col in ['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']):
        # Konversi bulan dari nama ke angka
        month_map = {
            'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
            'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
        }

        df['arrival_date_month_num'] = df['arrival_date_month'].map(month_map)

        # Membuat kolom tanggal kedatangan
        df['arrival_date'] = pd.to_datetime(
            df['arrival_date_year'].astype(str) + '-' +
            df['arrival_date_month_num'].astype(str) + '-' +
            df['arrival_date_day_of_month'].astype(str)
        )

        # Ekstrak fitur waktu tambahan
        df['arrival_year'] = df['arrival_date'].dt.year
        df['arrival_month'] = df['arrival_date'].dt.month_name()
        df['arrival_day'] = df['arrival_date'].dt.day
        df['arrival_weekday'] = df['arrival_date'].dt.day_name()
        df['arrival_quarter'] = df['arrival_date'].dt.quarter
        df['is_weekend_arrival'] = df['arrival_date'].dt.dayofweek >= 5  # 5 = Saturday, 6 = Sunday

        logger.info("Fitur tanggal kedatangan berhasil dibuat.")
        print("Membuat fitur tanggal kedatangan dan mengekstrak fitur waktu tambahan.")
        print("Alasan: Fitur waktu yang terstruktur memudahkan analisis pola musiman dan tren pemesanan.")

    # Total durasi menginap
    if 'stays_in_weekend_nights' in df.columns and 'stays_in_week_nights' in df.columns:
        df['total_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']
        logger.info("Fitur total_nights berhasil dibuat.")
        print("Membuat fitur total_nights untuk menghitung total durasi menginap.")
        print("Alasan: Memudahkan analisis durasi menginap tanpa perlu menjumlahkan dua kolom terpisah setiap kali.")

    # Kategori lead time
    if 'lead_time' in df.columns:
        df['lead_time_group'] = pd.cut(
            df['lead_time'],
            bins=[-1, 7, 30, 90, 180, 365, float('inf')],
            labels=['0-7 hari', '8-30 hari', '31-90 hari', '91-180 hari', '181-365 hari', '>365 hari']
        )
        logger.info("Fitur lead_time_group berhasil dibuat.")
        print("Membuat fitur lead_time_group untuk mengelompokkan lead time menjadi kategori yang lebih mudah dianalisis.")
        print("Alasan: Pengelompokan lead time memudahkan analisis pola pemesanan berdasarkan seberapa jauh di muka pemesanan dilakukan.")

    # Fitur boolean untuk mempermudah analisis
    if 'children' in df.columns and 'babies' in df.columns:
        df['has_children'] = ((df['children'] > 0) | (df['babies'] > 0)).astype(int)
        logger.info("Fitur has_children berhasil dibuat.")
        print("Membuat fitur has_children untuk menandai pemesanan dengan anak-anak.")
        print("Alasan: Memudahkan segmentasi tamu berdasarkan komposisi keluarga.")

    if 'booking_changes' in df.columns:
        df['has_booking_changes'] = (df['booking_changes'] > 0).astype(int)
        logger.info("Fitur has_booking_changes berhasil dibuat.")
        print("Membuat fitur has_booking_changes untuk menandai pemesanan yang mengalami perubahan.")
        print("Alasan: Memudahkan analisis perilaku perubahan pemesanan.")

    if 'customer_type' in df.columns:
        df['is_repeat_guest'] = (df['customer_type'] == 'Returning').astype(int)
        logger.info("Fitur is_repeat_guest berhasil dibuat.")
        print("Membuat fitur is_repeat_guest untuk menandai tamu yang kembali.")
        print("Alasan: Memudahkan analisis loyalitas tamu.")

    if 'market_segment' in df.columns:
        df['is_direct_booking'] = (df['market_segment'] == 'Direct').astype(int)
        logger.info("Fitur is_direct_booking berhasil dibuat.")
        print("Membuat fitur is_direct_booking untuk menandai pemesanan langsung.")
        print("Alasan: Memudahkan analisis saluran pemesanan.")

    if 'distribution_channel' in df.columns:
        df['is_online_booking'] = (df['distribution_channel'] == 'TA/TO').astype(int)
        logger.info("Fitur is_online_booking berhasil dibuat.")
        print("Membuat fitur is_online_booking untuk menandai pemesanan online.")
        print("Alasan: Memudahkan analisis saluran distribusi.")

    # Fitur untuk analisis tipe kamar
    if 'reserved_room_type' in df.columns and 'assigned_room_type' in df.columns:
        df['room_type_match'] = (df['reserved_room_type'] == df['assigned_room_type']).astype(int)

        # Menentukan apakah terjadi upgrade atau downgrade
        # Asumsi: A > B > C > ... (A adalah tipe kamar terbaik)
        df['room_type_upgrade'] = (
            (df['reserved_room_type'] > df['assigned_room_type']) &
            (df['reserved_room_type'] != df['assigned_room_type'])
        ).astype(int)

        df['room_type_downgrade'] = (
            (df['reserved_room_type'] < df['assigned_room_type']) &
            (df['reserved_room_type'] != df['assigned_room_type'])
        ).astype(int)

        logger.info("Fitur analisis tipe kamar berhasil dibuat.")
        print("Membuat fitur room_type_match, room_type_upgrade, dan room_type_downgrade untuk analisis tipe kamar.")
        print("Alasan: Memudahkan analisis kecocokan tipe kamar yang dipesan vs yang diberikan, serta upgrade dan downgrade.")

    # Fitur untuk analisis asal tamu
    if 'country' in df.columns:
        # Menambahkan kolom is_domestic (tamu dari Portugal)
        df['is_domestic'] = (df['country'] == 'PRT').astype(int)

        # Memeriksa nilai unik negara
        unique_countries = df['country'].unique()
        print(f"\nTerdapat {len(unique_countries)} negara unik dalam dataset.")

        # Mengelompokkan negara berdasarkan region
        # Daftar negara yang lebih lengkap berdasarkan region
        europe_countries = [
            'PRT', 'GBR', 'FRA', 'ESP', 'DEU', 'ITA', 'IRL', 'BEL', 'NLD', 'CHE', 'AUT', 'SWE', 'POL', 'GRC', 'DNK',
            'RUS', 'NOR', 'FIN', 'LUX', 'ROU', 'HRV', 'CZE', 'HUN', 'SVK', 'SVN', 'BGR', 'EST', 'LVA', 'LTU', 'CYP',
            'MLT', 'ISL', 'SRB', 'MKD', 'ALB', 'BIH', 'MDA', 'MNE', 'UKR', 'BLR', 'AND', 'LIE', 'MCO', 'SMR', 'VAT',
            'GIB', 'IMN', 'GGY', 'JEY', 'FRO', 'ALA', 'TUR'
        ]

        america_countries = [
            'USA', 'BRA', 'CAN', 'ARG', 'MEX', 'CHL', 'COL', 'URY', 'PER', 'VEN', 'ECU', 'BOL', 'PRY', 'GTM', 'CUB',
            'DOM', 'HTI', 'HND', 'SLV', 'NIC', 'CRI', 'PAN', 'JAM', 'TTO', 'BHS', 'BRB', 'GUY', 'SUR', 'BLZ', 'ATG',
            'DMA', 'GRD', 'KNA', 'LCA', 'VCT', 'ABW', 'CUW', 'SXM', 'BES', 'AIA', 'BMU', 'VGB', 'CYM', 'MSR', 'TCA',
            'PRI', 'VIR', 'GLP', 'MTQ', 'GUF'
        ]

        asia_pacific_countries = [
            'CHN', 'AUS', 'JPN', 'KOR', 'TWN', 'HKG', 'SGP', 'THA', 'MYS', 'PHL', 'IDN', 'IND', 'NZL', 'VNM', 'PAK',
            'BGD', 'NPL', 'LKA', 'MMR', 'KHM', 'LAO', 'BRN', 'MDV', 'BTN', 'TLS', 'MAC', 'FJI', 'PNG', 'SLB', 'VUT',
            'KIR', 'MHL', 'FSM', 'NRU', 'PLW', 'TON', 'WSM', 'TUV'
        ]

        middle_east_north_africa = [
            'ARE', 'SAU', 'IRN', 'ISR', 'QAT', 'KWT', 'BHR', 'OMN', 'JOR', 'LBN', 'IRQ', 'SYR', 'YEM', 'PSE', 'EGY',
            'MAR', 'DZA', 'TUN', 'LBY', 'SDN', 'SSD'
        ]

        africa_countries = [
            'ZAF', 'NGA', 'KEN', 'GHA', 'ETH', 'UGA', 'TZA', 'ZMB', 'ZWE', 'AGO', 'MOZ', 'NAM', 'BWA', 'SEN', 'MLI',
            'BFA', 'NER', 'TCD', 'CIV', 'GIN', 'BEN', 'TGO', 'LBR', 'SLE', 'GMB', 'GNB', 'CPV', 'MRT', 'CMR', 'GAB',
            'COG', 'COD', 'CAF', 'GNQ', 'STP', 'DJI', 'ERI', 'SOM', 'MDG', 'MUS', 'COM', 'SYC', 'RWA', 'BDI', 'LSO', 'SWZ'
        ]

        # Fungsi untuk mendapatkan region berdasarkan kode negara
        def get_region(country):
            if country in europe_countries:
                return 'Eropa'
            elif country in america_countries:
                return 'Amerika'
            elif country in asia_pacific_countries:
                return 'Asia Pasifik'
            elif country in middle_east_north_africa:
                return 'Timur Tengah & Afrika Utara'
            elif country in africa_countries:
                return 'Afrika'
            else:
                return 'Lainnya'

        # Menerapkan fungsi get_region ke kolom country
        df['region'] = df['country'].apply(get_region)

        # Memeriksa distribusi region
        region_counts = df['region'].value_counts()
        print("\nDistribusi region:")
        print(region_counts)

        # Memeriksa negara yang masuk kategori 'Lainnya'
        other_countries = df[df['region'] == 'Lainnya']['country'].unique()
        if len(other_countries) > 0:
            print(f"\nNegara yang masuk kategori 'Lainnya': {', '.join(other_countries)}")
            print("Catatan: Negara-negara ini perlu dikategorikan dengan benar jika jumlahnya signifikan.")

        logger.info("Fitur analisis asal tamu berhasil dibuat.")
        print("\nMembuat fitur is_domestic dan region untuk analisis asal tamu.")
        print("Alasan: Pengelompokan negara berdasarkan region geografis sangat penting untuk:")
        print("1. Memahami distribusi geografis tamu hotel")
        print("2. Mengidentifikasi pasar utama dan potensial")
        print("3. Menyesuaikan strategi pemasaran berdasarkan region")
        print("4. Menganalisis preferensi dan perilaku tamu berdasarkan asal geografis")
        print("5. Memvisualisasikan data dengan lebih efektif dalam analisis selanjutnya")

    return df

### 5.1 Menerapkan Pembuatan Fitur Baru

Sekarang kita akan menerapkan fungsi pembuatan fitur baru pada dataset.

In [10]:
if not hotel_df.empty:
    hotel_df = create_features(hotel_df)

    print("\n=== FITUR BARU YANG TELAH DIBUAT ===")
    new_features = [
        'arrival_date', 'arrival_month', 'arrival_weekday', 'is_weekend_arrival',
        'total_nights', 'lead_time_group', 'has_children', 'has_booking_changes',
        'is_repeat_guest', 'is_direct_booking', 'is_online_booking',
        'room_type_match', 'room_type_upgrade', 'room_type_downgrade',
        'is_domestic', 'region'
    ]

    for feature in new_features:
        if feature in hotel_df.columns:
            print(f"- {feature}")

    print("\n=== SAMPEL DATA DENGAN FITUR BARU ===")
    display(hotel_df.head())

2025-05-02 09:20:05,840 - hotel_preprocessing - INFO - Membuat fitur baru...
2025-05-02 09:20:05,974 - hotel_preprocessing - INFO - Fitur tanggal kedatangan berhasil dibuat.
2025-05-02 09:20:05,976 - hotel_preprocessing - INFO - Fitur total_nights berhasil dibuat.
2025-05-02 09:20:05,979 - hotel_preprocessing - INFO - Fitur lead_time_group berhasil dibuat.
2025-05-02 09:20:05,980 - hotel_preprocessing - INFO - Fitur has_children berhasil dibuat.
2025-05-02 09:20:05,980 - hotel_preprocessing - INFO - Fitur has_booking_changes berhasil dibuat.


Membuat fitur tanggal kedatangan dan mengekstrak fitur waktu tambahan.
Alasan: Fitur waktu yang terstruktur memudahkan analisis pola musiman dan tren pemesanan.
Membuat fitur total_nights untuk menghitung total durasi menginap.
Alasan: Memudahkan analisis durasi menginap tanpa perlu menjumlahkan dua kolom terpisah setiap kali.
Membuat fitur lead_time_group untuk mengelompokkan lead time menjadi kategori yang lebih mudah dianalisis.
Alasan: Pengelompokan lead time memudahkan analisis pola pemesanan berdasarkan seberapa jauh di muka pemesanan dilakukan.
Membuat fitur has_children untuk menandai pemesanan dengan anak-anak.
Alasan: Memudahkan segmentasi tamu berdasarkan komposisi keluarga.
Membuat fitur has_booking_changes untuk menandai pemesanan yang mengalami perubahan.
Alasan: Memudahkan analisis perilaku perubahan pemesanan.


2025-05-02 09:20:05,987 - hotel_preprocessing - INFO - Fitur is_repeat_guest berhasil dibuat.
2025-05-02 09:20:05,993 - hotel_preprocessing - INFO - Fitur is_direct_booking berhasil dibuat.
2025-05-02 09:20:06,000 - hotel_preprocessing - INFO - Fitur is_online_booking berhasil dibuat.


Membuat fitur is_repeat_guest untuk menandai tamu yang kembali.
Alasan: Memudahkan analisis loyalitas tamu.
Membuat fitur is_direct_booking untuk menandai pemesanan langsung.
Alasan: Memudahkan analisis saluran pemesanan.
Membuat fitur is_online_booking untuk menandai pemesanan online.
Alasan: Memudahkan analisis saluran distribusi.


2025-05-02 09:20:06,037 - hotel_preprocessing - INFO - Fitur analisis tipe kamar berhasil dibuat.
2025-05-02 09:20:06,081 - hotel_preprocessing - INFO - Fitur analisis asal tamu berhasil dibuat.


Membuat fitur room_type_match, room_type_upgrade, dan room_type_downgrade untuk analisis tipe kamar.
Alasan: Memudahkan analisis kecocokan tipe kamar yang dipesan vs yang diberikan, serta upgrade dan downgrade.

Terdapat 177 negara unik dalam dataset.

Distribusi region:
region
Eropa                          107959
Amerika                          4976
Asia Pasifik                     2338
Timur Tengah & Afrika Utara      1415
Lainnya                          1372
Afrika                            667
Name: count, dtype: int64

Negara yang masuk kategori 'Lainnya': CN, GEO, KAZ, AZE, ARM, MWI, UZB, TJK, TMP, UMI, MYT, PYF, ATA, ASM, NCL, ATF
Catatan: Negara-negara ini perlu dikategorikan dengan benar jika jumlahnya signifikan.

Membuat fitur is_domestic dan region untuk analisis asal tamu.
Alasan: Pengelompokan negara berdasarkan region geografis sangat penting untuk:
1. Memahami distribusi geografis tamu hotel
2. Mengidentifikasi pasar utama dan potensial
3. Menyesuaikan strategi pema

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,has_children,has_booking_changes,is_repeat_guest,is_direct_booking,is_online_booking,room_type_match,room_type_upgrade,room_type_downgrade,is_domestic,region
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,0,1,0,1,0,1,0,0,1,Eropa
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,0,1,0,1,0,1,0,0,1,Eropa
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,0,0,0,1,0,0,0,1,0,Eropa
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,0,0,0,0,0,1,0,0,0,Eropa
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,0,0,0,0,1,1,0,0,0,Eropa


## 6. Menyimpan Data yang Telah Diproses

Pada bagian ini, kita akan menyimpan dataset yang telah diproses untuk digunakan dalam analisis dan visualisasi selanjutnya. Kita juga akan membuat sampel dataset untuk visualisasi dengan ukuran yang lebih kecil untuk meningkatkan performa visualisasi.

In [11]:
def save_processed_data(df, output_path):
    """
    Menyimpan dataset yang telah diproses.

    Parameters:
    -----------
    df : pandas.DataFrame
        Dataset yang telah diproses
    output_path : str
        Path untuk menyimpan dataset
    """
    logger.info(f"Menyimpan dataset yang telah diproses ke {output_path}")
    df.to_csv(output_path, index=False)
    logger.info(f"Dataset berhasil disimpan dengan {df.shape[0]} baris dan {df.shape[1]} kolom.")

def create_visualization_sample(df, output_path, sample_size=10000):
    """
    Membuat sampel dataset untuk visualisasi.

    Parameters:
    -----------
    df : pandas.DataFrame
        Dataset lengkap
    output_path : str
        Path untuk menyimpan sampel dataset
    sample_size : int, optional
        Ukuran sampel, default 10000
    """
    if len(df) > sample_size:
        # Stratified sampling berdasarkan hotel dan is_canceled
        if 'hotel' in df.columns and 'is_canceled' in df.columns:
            df_sample = df.groupby(['hotel', 'is_canceled']).apply(
                lambda x: x.sample(min(len(x), int(sample_size * len(x) / len(df))))
            ).reset_index(drop=True)
        else:
            df_sample = df.sample(sample_size, random_state=42)

        # Jika ukuran sampel masih terlalu besar, lakukan random sampling
        if len(df_sample) > sample_size:
            df_sample = df_sample.sample(sample_size, random_state=42)
    else:
        df_sample = df.copy()

    logger.info(f"Membuat sampel dataset untuk visualisasi dengan {len(df_sample)} baris.")
    df_sample.to_csv(output_path, index=False)
    logger.info(f"Sampel dataset berhasil disimpan ke {output_path}.")

### 6.1 Menyimpan Dataset yang Telah Diproses

Sekarang kita akan menyimpan dataset yang telah diproses dan membuat sampel dataset untuk visualisasi.

In [12]:
if not hotel_df.empty:
    # Path untuk menyimpan dataset
    processed_data_path = os.path.join(DATA_DIR, 'hotel_bookings_processed.csv')
    viz_sample_path = os.path.join(DATA_DIR, 'hotel_bookings_viz.csv')

    # Menyimpan dataset yang telah diproses
    save_processed_data(hotel_df, processed_data_path)
    print(f"Dataset yang telah diproses berhasil disimpan ke {processed_data_path}")

    # Membuat sampel dataset untuk visualisasi
    create_visualization_sample(hotel_df, viz_sample_path)
    print(f"Sampel dataset untuk visualisasi berhasil disimpan ke {viz_sample_path}")

2025-05-02 09:20:06,100 - hotel_preprocessing - INFO - Menyimpan dataset yang telah diproses ke hotel_booking_data/hotel_bookings_processed.csv
2025-05-02 09:20:07,372 - hotel_preprocessing - INFO - Dataset berhasil disimpan dengan 118727 baris dan 52 kolom.
2025-05-02 09:20:07,475 - hotel_preprocessing - INFO - Membuat sampel dataset untuk visualisasi dengan 9998 baris.


Dataset yang telah diproses berhasil disimpan ke hotel_booking_data/hotel_bookings_processed.csv


2025-05-02 09:20:07,614 - hotel_preprocessing - INFO - Sampel dataset berhasil disimpan ke hotel_booking_data/hotel_bookings_viz.csv.


Sampel dataset untuk visualisasi berhasil disimpan ke hotel_booking_data/hotel_bookings_viz.csv


## 7. Ringkasan Preprocessing

Pada bagian ini, kita akan meringkas langkah-langkah preprocessing yang telah dilakukan dan memberikan gambaran umum tentang dataset yang telah diproses.

In [13]:
if not hotel_df.empty:
    print("\n=== RINGKASAN PREPROCESSING ===")
    print(f"1. Ukuran dataset awal: {hotel_df.shape[0]} baris, {hotel_df.shape[1]} kolom")

    print("\n2. Langkah-langkah preprocessing yang telah dilakukan:")
    print("   - Penanganan missing values")
    print("   - Pembersihan data dari nilai yang tidak valid atau inkonsisten")
    print("   - Pembuatan fitur baru untuk analisis")

    print("\n3. Fitur-fitur baru yang telah dibuat:")
    for feature in new_features:
        if feature in hotel_df.columns:
            print(f"   - {feature}")

    print("\n4. Distribusi kategorikal:")
    categorical_columns = ['hotel', 'meal', 'market_segment', 'distribution_channel',
                          'reserved_room_type', 'customer_type', 'deposit_type', 'region']

    for col in categorical_columns:
        if col in hotel_df.columns:
            print(f"\n{col}:")
            display(hotel_df[col].value_counts().head())

    print("\n5. Statistik deskriptif untuk kolom numerik:")
    numeric_columns = ['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights',
                       'adults', 'children', 'babies', 'total_nights']

    display(hotel_df[numeric_columns].describe())

    print("\n6. Dataset yang telah diproses telah disimpan untuk analisis dan visualisasi selanjutnya.")


=== RINGKASAN PREPROCESSING ===
1. Ukuran dataset awal: 118727 baris, 52 kolom

2. Langkah-langkah preprocessing yang telah dilakukan:
   - Penanganan missing values
   - Pembersihan data dari nilai yang tidak valid atau inkonsisten
   - Pembuatan fitur baru untuk analisis

3. Fitur-fitur baru yang telah dibuat:
   - arrival_date
   - arrival_month
   - arrival_weekday
   - is_weekend_arrival
   - total_nights
   - lead_time_group
   - has_children
   - has_booking_changes
   - is_repeat_guest
   - is_direct_booking
   - is_online_booking
   - room_type_match
   - room_type_upgrade
   - room_type_downgrade
   - is_domestic
   - region

4. Distribusi kategorikal:

hotel:


hotel
City Hotel      79143
Resort Hotel    39584
Name: count, dtype: int64


meal:


meal
BB           91789
HB           14428
SC           10547
Undefined     1165
FB             798
Name: count, dtype: int64


market_segment:


...


region:


region
Eropa                          107959
Amerika                          4976
Asia Pasifik                     2338
Timur Tengah & Afrika Utara      1415
Lainnya                          1372
Name: count, dtype: int64


5. Statistik deskriptif untuk kolom numerik:


,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,total_nights
count,118727.000000,118727.000000,118727.000000,118727.000000,118727.000000,118727.000000,118727.000000
mean,104.401324,0.928298,2.500914,1.861051,0.104349,0.007959,3.429212
std,106.915734,0.992724,1.889096,0.574699,0.399432,0.097450,2.528321
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,0.000000,1.000000,2.000000,0.000000,0.000000,2.000000
50%,70.000000,1.000000,2.000000,2.000000,0.000000,0.000000,3.000000
75%,161.000000,2.000000,3.000000,2.000000,0.000000,0.000000,4.000000
max,737.000000,16.000000,40.000000,55.000000,10.000000,10.000000,56.000000



6. Dataset yang telah diproses telah disimpan untuk analisis dan visualisasi selanjutnya.


## 8. Kesimpulan

Pada notebook ini, kita telah melakukan preprocessing data pemesanan hotel untuk persiapan analisis dan visualisasi. Langkah-langkah preprocessing yang telah dilakukan meliputi:

1. **Penanganan missing values**: Menghapus kolom dengan missing values tinggi, menghapus baris dengan missing values pada kolom penting, dan mengisi missing values dengan nilai yang sesuai.
2. **Pembersihan data**: Menghapus nilai yang tidak valid atau inkonsisten, seperti nilai undefined di distribution_channel, reservasi tanpa tamu, dan memperbaiki inkonsistensi antara is_canceled dan reservation_status.
3. **Pembuatan fitur baru**: Membuat fitur baru yang berguna untuk analisis, seperti fitur tanggal, durasi menginap, kategori lead time, fitur boolean, fitur analisis tipe kamar, dan fitur analisis asal tamu.

### Pentingnya Kategorisasi Negara yang Tepat

Salah satu aspek penting dalam preprocessing ini adalah kategorisasi negara berdasarkan region geografis. Hal ini penting karena:

1. **Memahami Distribusi Geografis Tamu**: Dengan mengelompokkan negara ke dalam region, kita dapat memahami dari mana tamu hotel berasal secara lebih terstruktur.

2. **Mengidentifikasi Pasar Utama dan Potensial**: Analisis berdasarkan region membantu mengidentifikasi pasar utama yang sudah ada dan pasar potensial yang belum dioptimalkan.

3. **Strategi Pemasaran yang Ditargetkan**: Informasi region memungkinkan hotel untuk mengembangkan strategi pemasaran yang ditargetkan untuk region tertentu, dengan mempertimbangkan perbedaan budaya, preferensi, dan perilaku pemesanan.

4. **Analisis Musiman**: Pola pemesanan dari region yang berbeda mungkin memiliki tren musiman yang berbeda, yang dapat dimanfaatkan untuk strategi harga dan kapasitas.

5. **Visualisasi yang Lebih Efektif**: Mengelompokkan negara ke dalam region membuat visualisasi data menjadi lebih sederhana dan informatif, dibandingkan dengan menampilkan setiap negara secara terpisah.

6. **Analisis Preferensi**: Region yang berbeda mungkin memiliki preferensi yang berbeda dalam hal tipe kamar, durasi menginap, atau layanan tambahan, yang dapat digunakan untuk pengembangan produk dan layanan.

Dengan preprocessing yang telah dilakukan, dataset kini siap untuk analisis dan visualisasi lanjutan yang akan memberikan insight berharga untuk pengambilan keputusan bisnis.